# 1. Feature Engineering (Obtained before hand)

In [1]:
#reading data :)
import numpy as np
import pandas as pd
train=pd.read_csv('train.csv')
test =pd.read_csv('test.csv')
test['target']=-1 # just to match columns
both=test.copy()
both=both.append(train.copy())
del test  #bye!
del train #bye!
cols=both.columns.drop(['id','target']).tolist()
print('columns: ',both.columns.tolist())
print('target values: ',both['target'].unique())

columns:  ['id', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11', 'ps_car_11_cat', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'target']
target values:  [-1  0  1]


In [2]:
noncat=[]
cats=[]
cats_prefix={}
for i in cols:
    unique_train=both[both['target']!=-1][i].unique()
    unique_both =both[i].unique()
    equal=(sorted(unique_train) == sorted(unique_both))
    length_train=len(unique_train)
    length_both=len(unique_both)
    print('Column: ',i,'\t unique values at train/both=',
          length_train,' / ',length_both,
          '\t <- categorical?!' if equal else ''
         )
    if(not equal):
        noncat.append(i)
    if(equal and length_both>2):
        cats.append(i)
        cats_prefix[i]="OHE_"+i
print("these variables should be categorical, or not?! =) ",cats)

Column:  ps_calc_01 	 unique values at train/both= 10  /  10 	 <- categorical?!
Column:  ps_calc_02 	 unique values at train/both= 10  /  10 	 <- categorical?!
Column:  ps_calc_03 	 unique values at train/both= 10  /  10 	 <- categorical?!
Column:  ps_calc_04 	 unique values at train/both= 6  /  6 	 <- categorical?!
Column:  ps_calc_05 	 unique values at train/both= 7  /  7 	 <- categorical?!
Column:  ps_calc_06 	 unique values at train/both= 11  /  11 	 <- categorical?!
Column:  ps_calc_07 	 unique values at train/both= 10  /  10 	 <- categorical?!
Column:  ps_calc_08 	 unique values at train/both= 11  /  12 
Column:  ps_calc_09 	 unique values at train/both= 8  /  8 	 <- categorical?!
Column:  ps_calc_10 	 unique values at train/both= 26  /  26 	 <- categorical?!
Column:  ps_calc_11 	 unique values at train/both= 20  /  21 
Column:  ps_calc_12 	 unique values at train/both= 11  /  12 
Column:  ps_calc_13 	 unique values at train/both= 14  /  16 
Column:  ps_calc_14 	 unique values at

In [3]:
#i will OHE to you :)
both=pd.get_dummies(both,prefix=cats_prefix,columns=cats)
both[both['target']!=-1].to_csv('train.cat.ohe.csv.gzip',index=False,compression='gzip')
both[both['target']==-1].to_csv('test.cat.ohe.csv.gzip' ,index=False,compression='gzip')

print('features:',len(both.columns.drop(['target','id']).tolist()))

features: 395


In [4]:
del both
import gc
gc.collect()

119

keras
from: https://www.kaggle.com/tilii7/keras-averaging-runs-gini-early-stopping

In [5]:
from datetime import datetime
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [7]:
class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod(
            (datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' %
              (thour, tmin, round(tsec, 2)))

def scale_data(X, scaler=None):
    if not scaler:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler

In [8]:
# train and test data path
DATA_TRAIN_PATH = 'train.cat.ohe.csv.gzip'
DATA_TEST_PATH = 'test.cat.ohe.csv.gzip'

def load_data(path_train=DATA_TRAIN_PATH, path_test=DATA_TEST_PATH):
    train_loader = pd.read_csv(path_train, compression='gzip', 
                               dtype={'target': np.int8, 'id': np.int32})
    train = train_loader.drop(['target', 'id'], axis=1)
    train_labels = train_loader['target'].values
    train_ids = train_loader['id'].values
    print('\n Shape of raw train data:', train.shape)

    test_loader = pd.read_csv(path_test, compression='gzip', dtype={'id': np.int32})
    
    test = test_loader.drop(['id','target'], axis=1)
    test_ids = test_loader['id'].values
    print(' Shape of raw test data:', test.shape)

    return train, train_labels, test, train_ids, test_ids

In [9]:
folds = 4
runs = 2

cv_LL = 0
cv_AUC = 0
cv_gini = 0
fpred = []
avpred = []
avreal = []
avids = []

In [10]:
# Load data set and target values
train, target, test, tr_ids, te_ids = load_data()
n_train = train.shape[0]
train_test = pd.concat((train, test)).reset_index(drop=True)
train_test_scaled, scaler = scale_data(train_test)
train = train_test_scaled[:n_train, :]
test = train_test_scaled[n_train:, :]
print('\n Shape of processed train data:', train.shape)
print(' Shape of processed test data:', test.shape)


 Shape of raw train data: (595212, 395)
 Shape of raw test data: (892816, 395)

 Shape of processed train data: (595212, 395)
 Shape of processed test data: (892816, 395)


In [11]:
patience = 10

batchsize = 128

In [13]:
%%time
# Let's split the data into folds. I always use the same random number for reproducibility, 
# and suggest that you do the same (you certainly don't have to use 1001).

skf = StratifiedKFold(n_splits=folds, random_state=1001)
starttime = timer(None)
for i, (train_index, test_index) in enumerate(skf.split(train, target)):
    start_time = timer(None)
    X_train, X_val = train[train_index], train[test_index]
    y_train, y_val = target[train_index], target[test_index]
    train_ids, val_ids = tr_ids[train_index], tr_ids[test_index]
    
# This is where we define and compile the model. These parameters are not optimal, as they were chosen 
# to get a notebook to complete in 60 minutes. Other than leaving BatchNormalization and last sigmoid 
# activation alone, virtually everything else can be optimized: number of neurons, types of initializers, 
# activation functions, dropout values. The same goes for the optimizer at the end.

#########
# Never move this model definition to the beginning of the file or anywhere else outside of this loop. 
# The model needs to be initialized anew every time you run a different fold. If not, it will continue 
# the training from a previous model, and that is not what you want.
#########

    # This definition must be within the for loop or else it will continue training previous model
    def baseline_model():
        model = Sequential()
        model.add(
            Dense(
                200,
                input_dim=X_train.shape[1],
                kernel_initializer='glorot_normal',
                ))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(100, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(50, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.15))
        model.add(Dense(25, kernel_initializer='glorot_normal'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.1))
        model.add(Dense(1, activation='sigmoid'))

        # Compile model
        model.compile(optimizer='adam', metrics = ['accuracy'], loss='binary_crossentropy')

        return model

# This is where we repeat the runs for each fold. If you choose runs=1 above, it will run a 
# regular N-fold procedure.

#########
# It is important to leave the call to random seed here, so each run starts with a different seed.
#########

    for run in range(runs):
        print('\n Fold %d - Run %d\n' % ((i + 1), (run + 1)))
        np.random.seed()

# Lots to unpack here.

# The first callback prints out roc_auc and gini values at the end of each epoch. It must be listed 
# before the EarlyStopping callback, which monitors gini values saved in the previous callback. Make 
# sure to set the mode to "max" because the default value ("auto") will not handle gini properly 
# (it will act as if the model is not improving even when roc/gini go up).

# CSVLogger creates a record of all iterations. Not really needed but it doesn't hurt to have it.

# ModelCheckpoint saves a model each time gini improves. Its mode also must be set to "max" for reasons 
# explained above.

        callbacks = [
            roc_auc_callback(training_data=(X_train, y_train),validation_data=(X_val, y_val)),  # call this before EarlyStopping
            EarlyStopping(monitor='norm_gini_val', patience=patience, mode='max', verbose=1),
            CSVLogger('keras-5fold-run-01-v1-epochs.log', separator=',', append=False),
            ModelCheckpoint(
                    'keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check',
                    monitor='norm_gini_val', mode='max', # mode must be set to max or Keras will be confused
                    save_best_only=True,
                    verbose=1)
        ]

# The classifier is defined here. Epochs should be be set to a very large number (not 3 like below) which 
# will never be reached anyway because of early stopping. I usually put 5000 there. Because why not.

        nnet = KerasClassifier(
            build_fn=baseline_model,
# Epoch needs to be set to a very large number ; early stopping will prevent it from reaching
#            epochs=5000,
            epochs=5000,
            batch_size=batchsize,
            validation_data=(X_val, y_val),
            verbose=2,
            shuffle=True,
            callbacks=callbacks)

        fit = nnet.fit(X_train, y_train)
        
# We want the best saved model - not the last one where the training stopped. So we delete the old 
# model instance and load the model from the last saved checkpoint. Next we predict values both for 
# validation and test data, and create a summary of parameters for each run.

        del nnet
        nnet = load_model('keras-5fold-run-01-v1-fold-' + str('%02d' % (i + 1)) + '-run-' + str('%02d' % (run + 1)) + '.check')
        scores_val_run = nnet.predict_proba(X_val, verbose=0)
        LL_run = log_loss(y_val, scores_val_run)
        print('\n Fold %d Run %d Log-loss: %.5f' % ((i + 1), (run + 1), LL_run))
        AUC_run = roc_auc_score(y_val, scores_val_run)
        print(' Fold %d Run %d AUC: %.5f' % ((i + 1), (run + 1), AUC_run))
        print(' Fold %d Run %d normalized gini: %.5f' % ((i + 1), (run + 1), AUC_run*2-1))
        y_pred_run = nnet.predict_proba(test, verbose=0)
        if run > 0:
            scores_val = scores_val + scores_val_run
            y_pred = y_pred + y_pred_run
        else:
            scores_val = scores_val_run
            y_pred = y_pred_run
            
# We average all runs from the same fold and provide a parameter summary for each fold. Unless something 
# is wrong, the numbers printed here should be better than any of the individual runs.

    scores_val = scores_val / runs
    y_pred = y_pred / runs
    LL = log_loss(y_val, scores_val)
    print('\n Fold %d Log-loss: %.5f' % ((i + 1), LL))
    AUC = roc_auc_score(y_val, scores_val)
    print(' Fold %d AUC: %.5f' % ((i + 1), AUC))
    print(' Fold %d normalized gini: %.5f' % ((i + 1), AUC*2-1))
    timer(start_time)
    
# We add up predictions on the test data for each fold. Create out-of-fold predictions for validation data.

    if i > 0:
        fpred = pred + y_pred
        avreal = np.concatenate((avreal, y_val), axis=0)
        avpred = np.concatenate((avpred, scores_val), axis=0)
        avids = np.concatenate((avids, val_ids), axis=0)
    else:
        fpred = y_pred
        avreal = y_val
        avpred = scores_val
        avids = val_ids
    pred = fpred
    cv_LL = cv_LL + LL
    cv_AUC = cv_AUC + AUC
    cv_gini = cv_gini + (AUC*2-1)


 Fold 1 - Run 1

Train on 446408 samples, validate on 148804 samples
Epoch 1/5000
roc_auc: 0.63058 - roc_auc_val: 0.62657 - norm_gini: 0.26117 - norm_gini_val: 0.25315          
Epoch 00000: norm_gini_val improved from -inf to 0.25315, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
62s - loss: 0.1839 - acc: 0.9527 - val_loss: 0.1529 - val_acc: 0.9635
Epoch 2/5000
roc_auc: 0.64 - roc_auc_val: 0.63115 - norm_gini: 0.27999 - norm_gini_val: 0.26231          
Epoch 00001: norm_gini_val improved from 0.25315 to 0.26231, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
56s - loss: 0.1550 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9635
Epoch 3/5000
roc_auc: 0.64327 - roc_auc_val: 0.63382 - norm_gini: 0.28654 - norm_gini_val: 0.26765          
Epoch 00002: norm_gini_val improved from 0.26231 to 0.26765, saving model to keras-5fold-run-01-v1-fold-01-run-01.check
55s - loss: 0.1538 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
Epoch 4/5000
roc_auc: 0.64669 - roc_auc

65s - loss: 0.1843 - acc: 0.9525 - val_loss: 0.1533 - val_acc: 0.9635
Epoch 2/5000
roc_auc: 0.64024 - roc_auc_val: 0.6306 - norm_gini: 0.28048 - norm_gini_val: 0.26119          
Epoch 00001: norm_gini_val improved from 0.23850 to 0.26119, saving model to keras-5fold-run-01-v1-fold-02-run-01.check
61s - loss: 0.1549 - acc: 0.9636 - val_loss: 0.1527 - val_acc: 0.9635
Epoch 3/5000
roc_auc: 0.64365 - roc_auc_val: 0.63066 - norm_gini: 0.2873 - norm_gini_val: 0.26131          
Epoch 00002: norm_gini_val improved from 0.26119 to 0.26131, saving model to keras-5fold-run-01-v1-fold-02-run-01.check
58s - loss: 0.1537 - acc: 0.9636 - val_loss: 0.1528 - val_acc: 0.9635
Epoch 4/5000
roc_auc: 0.64688 - roc_auc_val: 0.6322 - norm_gini: 0.29376 - norm_gini_val: 0.26439          
Epoch 00003: norm_gini_val improved from 0.26131 to 0.26439, saving model to keras-5fold-run-01-v1-fold-02-run-01.check
56s - loss: 0.1532 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9635
Epoch 5/5000
roc_auc: 0.64988 - roc_

68s - loss: 0.1846 - acc: 0.9522 - val_loss: 0.1531 - val_acc: 0.9636
Epoch 2/5000
roc_auc: 0.64145 - roc_auc_val: 0.63013 - norm_gini: 0.28289 - norm_gini_val: 0.26026          
Epoch 00001: norm_gini_val improved from 0.24314 to 0.26026, saving model to keras-5fold-run-01-v1-fold-03-run-01.check
63s - loss: 0.1547 - acc: 0.9635 - val_loss: 0.1526 - val_acc: 0.9636
Epoch 3/5000
roc_auc: 0.64444 - roc_auc_val: 0.63473 - norm_gini: 0.28889 - norm_gini_val: 0.26945          
Epoch 00002: norm_gini_val improved from 0.26026 to 0.26945, saving model to keras-5fold-run-01-v1-fold-03-run-01.check
63s - loss: 0.1539 - acc: 0.9636 - val_loss: 0.1526 - val_acc: 0.9636
Epoch 4/5000
roc_auc: 0.64721 - roc_auc_val: 0.63227 - norm_gini: 0.29442 - norm_gini_val: 0.26455          
Epoch 00003: norm_gini_val did not improve
63s - loss: 0.1532 - acc: 0.9636 - val_loss: 0.1525 - val_acc: 0.9636
Epoch 5/5000
roc_auc: 0.65207 - roc_auc_val: 0.63671 - norm_gini: 0.30414 - norm_gini_val: 0.27343          
E


 Fold 4 - Run 1

Train on 446410 samples, validate on 148802 samples
Epoch 1/5000
roc_auc: 0.63352 - roc_auc_val: 0.62177 - norm_gini: 0.26705 - norm_gini_val: 0.24354          
Epoch 00000: norm_gini_val improved from -inf to 0.24354, saving model to keras-5fold-run-01-v1-fold-04-run-01.check
71s - loss: 0.1846 - acc: 0.9517 - val_loss: 0.1532 - val_acc: 0.9636
Epoch 2/5000
roc_auc: 0.64063 - roc_auc_val: 0.62614 - norm_gini: 0.28126 - norm_gini_val: 0.25229          
Epoch 00001: norm_gini_val improved from 0.24354 to 0.25229, saving model to keras-5fold-run-01-v1-fold-04-run-01.check
66s - loss: 0.1547 - acc: 0.9635 - val_loss: 0.1527 - val_acc: 0.9636
Epoch 3/5000
roc_auc: 0.64593 - roc_auc_val: 0.62776 - norm_gini: 0.29186 - norm_gini_val: 0.25553          
Epoch 00002: norm_gini_val improved from 0.25229 to 0.25553, saving model to keras-5fold-run-01-v1-fold-04-run-01.check
64s - loss: 0.1535 - acc: 0.9635 - val_loss: 0.1527 - val_acc: 0.9636
Epoch 4/5000
roc_auc: 0.64752 - roc_

roc_auc: 0.674 - roc_auc_val: 0.62837 - norm_gini: 0.348 - norm_gini_val: 0.25673          
Epoch 00016: norm_gini_val did not improve
63s - loss: 0.1513 - acc: 0.9636 - val_loss: 0.1531 - val_acc: 0.9636
Epoch 18/5000
roc_auc: 0.67475 - roc_auc_val: 0.62856 - norm_gini: 0.3495 - norm_gini_val: 0.25711          
Epoch 00017: norm_gini_val did not improve
62s - loss: 0.1510 - acc: 0.9636 - val_loss: 0.1528 - val_acc: 0.9636
Epoch 19/5000
roc_auc: 0.67717 - roc_auc_val: 0.62633 - norm_gini: 0.35435 - norm_gini_val: 0.25265          
Epoch 00018: norm_gini_val did not improve
62s - loss: 0.1511 - acc: 0.9636 - val_loss: 0.1533 - val_acc: 0.9636
Epoch 00018: early stopping

 Fold 4 Run 2 Log-loss: 0.15257
 Fold 4 Run 2 AUC: 0.63184
 Fold 4 Run 2 normalized gini: 0.26368

 Fold 4 Log-loss: 0.15244
 Fold 4 AUC: 0.63242
 Fold 4 normalized gini: 0.26484

 Time taken: 0 hours 41 minutes and 45.71 seconds.
Wall time: 2h 26min 15s


In [14]:
LL_oof = log_loss(avreal, avpred)
print('\n Average Log-loss: %.5f' % (cv_LL/folds))
print(' Out-of-fold Log-loss: %.5f' % LL_oof)
AUC_oof = roc_auc_score(avreal, avpred)
print('\n Average AUC: %.5f' % (cv_AUC/folds))
print(' Out-of-fold AUC: %.5f' % AUC_oof)
print('\n Average normalized gini: %.5f' % (cv_gini/folds))
print(' Out-of-fold normalized gini: %.5f' % (AUC_oof*2-1))
score = str(round((AUC_oof*2-1), 5))
timer(starttime)
mpred = pred / folds


 Average Log-loss: 0.26650
 Out-of-fold Log-loss: 0.15225

 Average AUC: 1.11236
 Out-of-fold AUC: 0.63502

 Average normalized gini: 0.47471
 Out-of-fold normalized gini: 0.27005

 Time taken: 2 hours 54 minutes and 4.02 seconds.


In [15]:
print('#\n Writing results')
now = datetime.now()
oof_result = pd.DataFrame(avreal, columns=['target'])
oof_result['prediction'] = avpred
oof_result['id'] = avids
oof_result.sort_values('id', ascending=True, inplace=True)
oof_result = oof_result.set_index('id')
sub_file = 'train_5fold-keras-run-01-v1-oof_' + str(score) + '_' + str(now.strftime('%Y-%m-%d-%H-%M')) + '.csv'
print('\n Writing out-of-fold file:  %s' % sub_file)
oof_result.to_csv(sub_file, index=True, index_label='id')

#
 Writing results

 Writing out-of-fold file:  train_5fold-keras-run-01-v1-oof_0.27005_2017-10-31-16-24.csv


In [17]:
testnn=pd.read_csv('train_5fold-keras-run-01-v1-oof_0.27005_2017-10-31-16-24.csv')
testnn.shape

(595212, 3)

In [16]:
result = pd.DataFrame(mpred, columns=['target'])
result['id'] = te_ids
result = result.set_index('id')
print('\n First 10 lines of your 5-fold average prediction:\n')
print(result.head(10))
sub_file = 'submission_5fold-average-keras-run-01-v1_' + str(score) + '_' + str(now.strftime('%Y-%m-%d-%H-%M')) + '.csv'
print('\n Writing submission:  %s' % sub_file)
result.to_csv(sub_file, index=True, index_label='id')
result.head


 First 10 lines of your 5-fold average prediction:

      target
id          
0   0.030242
1   0.033049
2   0.022802
3   0.014566
4   0.036075
5   0.049619
6   0.016556
8   0.028519
10  0.057189
11  0.045363

 Writing submission:  submission_5fold-average-keras-run-01-v1_0.27005_2017-10-31-16-24.csv


<bound method NDFrame.head of            target
id               
0        0.030242
1        0.033049
2        0.022802
3        0.014566
4        0.036075
5        0.049619
6        0.016556
8        0.028519
10       0.057189
11       0.045363
12       0.031533
14       0.020338
15       0.047238
18       0.049369
21       0.047132
23       0.025002
24       0.025436
25       0.060219
27       0.017686
29       0.053053
30       0.042283
31       0.038353
32       0.045377
33       0.014678
37       0.031383
38       0.040908
39       0.071888
40       0.033456
41       0.034114
42       0.026141
...           ...
1487982  0.016875
1487984  0.038118
1487985  0.035744
1487986  0.030147
1487987  0.030141
1487989  0.026660
1487991  0.033473
1487993  0.033779
1487995  0.030496
1487997  0.033492
1487998  0.064389
1487999  0.068787
1488000  0.043493
1488002  0.071875
1488003  0.027533
1488004  0.034128
1488006  0.025110
1488007  0.030222
1488010  0.044069
1488012  0.039086
1488014  0.01980

In [18]:
#submission_5fold-average-keras-run-01-v1_0.27005_2017-10-31-16-24.csv
trainnn=pd.read_csv('submission_5fold-average-keras-run-01-v1_0.27005_2017-10-31-16-24.csv')
trainnn.shape

(892816, 2)

This two csv files should be easily converted to those required by stack model
after submission we have gini-score as around 0.269